In [3]:
#mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install pyunpack
!pip install patool
!pip install scipy==1.1.0

In [0]:
import warnings
warnings.filterwarnings("ignore")

import os
import random
import cv2
import math
import numpy as np
import scipy
import scipy.misc
from scipy import pi
from subprocess import call
from datetime import datetime
from itertools import islice
import matplotlib.pyplot as plt 
import tensorflow as tf

In [0]:
from pyunpack import Archive
Archive('drive/My Drive/Autopilot-TensorFlow-master.rar').extractall('Data')

In [0]:
#preparing The Dataset
import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


In [9]:
print(num_train_images)
print(num_val_images)

31784
13621


In [10]:
#model_architecture
import tensorflow as tf
import scipy

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
#h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=0.5)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

#h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob=0.5)


#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.tanh(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the atan output

W0814 19:25:58.819151 139790696757120 deprecation.py:506] From <ipython-input-10-7e66226e7030>:59: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
#Training the model with RMSPropOptimzer with 1e-4 learning rate
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2

LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.RMSPropOptimizer(1e-4).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(num_images/batch_size)):
    xs, ys = LoadTrainBatch(batch_size)
    train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.7})
    if i % 10 == 0:
      xs, ys = LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


W0814 19:26:08.745205 139790696757120 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0814 19:26:08.962972 139790696757120 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
W0814 19:26:17.021427 139790696757120 saver.py:1134] *******************************************************
W0814 19:26:17.027651 139790696757120 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W0814 19:26:17.029453 139790696757120 saver.py:1136] Cons

Epoch: 0, Step: 0, Loss: 8.63339
Epoch: 0, Step: 10, Loss: 8.01928
Epoch: 0, Step: 20, Loss: 7.36743
Epoch: 0, Step: 30, Loss: 7.1731
Epoch: 0, Step: 40, Loss: 6.75372
Epoch: 0, Step: 50, Loss: 6.56871
Epoch: 0, Step: 60, Loss: 6.6426
Epoch: 0, Step: 70, Loss: 6.98294
Epoch: 0, Step: 80, Loss: 6.68586
Epoch: 0, Step: 90, Loss: 6.31866


W0814 19:27:00.585821 139790696757120 saver.py:1134] *******************************************************
W0814 19:27:00.586978 139790696757120 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W0814 19:27:00.588450 139790696757120 saver.py:1136] Consider switching to the more efficient V2 format:
W0814 19:27:00.590055 139790696757120 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W0814 19:27:00.592093 139790696757120 saver.py:1138] now on by default.
W0814 19:27:00.595180 139790696757120 saver.py:1139] *******************************************************


Epoch: 0, Step: 100, Loss: 6.27794
Epoch: 0, Step: 110, Loss: 6.19002
Epoch: 0, Step: 120, Loss: 6.19399
Epoch: 0, Step: 130, Loss: 6.53036
Epoch: 0, Step: 140, Loss: 7.04063
Epoch: 0, Step: 150, Loss: 6.38338
Epoch: 0, Step: 160, Loss: 7.07707
Epoch: 0, Step: 170, Loss: 6.34055
Epoch: 0, Step: 180, Loss: 6.63367
Epoch: 0, Step: 190, Loss: 6.28891


W0814 19:27:42.315130 139790696757120 saver.py:1134] *******************************************************
W0814 19:27:42.316096 139790696757120 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W0814 19:27:42.316954 139790696757120 saver.py:1136] Consider switching to the more efficient V2 format:
W0814 19:27:42.321685 139790696757120 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W0814 19:27:42.323593 139790696757120 saver.py:1138] now on by default.
W0814 19:27:42.325153 139790696757120 saver.py:1139] *******************************************************


...

Epoch: 29, Step: 3100, Loss: 0.286158
Epoch: 29, Step: 3110, Loss: 0.151066
Epoch: 29, Step: 3120, Loss: 0.069472
Epoch: 29, Step: 3130, Loss: 0.109618
Epoch: 29, Step: 3140, Loss: 0.0680567
Epoch: 29, Step: 3150, Loss: 0.0672533
Epoch: 29, Step: 3160, Loss: 0.101867
Epoch: 29, Step: 3170, Loss: 0.0850564
Epoch: 29, Step: 3180, Loss: 0.416104
Epoch: 29, Step: 3190, Loss: 0.0549816


W0814 20:57:08.095653 139790696757120 saver.py:1134] *******************************************************
W0814 20:57:08.096679 139790696757120 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W0814 20:57:08.097442 139790696757120 saver.py:1136] Consider switching to the more efficient V2 format:
W0814 20:57:08.100753 139790696757120 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W0814 20:57:08.103043 139790696757120 saver.py:1138] now on by default.
W0814 20:57:08.105119 139790696757120 saver.py:1139] *******************************************************


Epoch: 29, Step: 3200, Loss: 0.0859954
Epoch: 29, Step: 3210, Loss: 0.0873268
Epoch: 29, Step: 3220, Loss: 0.131833
Epoch: 29, Step: 3230, Loss: 0.394799
Epoch: 29, Step: 3240, Loss: 0.0669598
Epoch: 29, Step: 3250, Loss: 0.0792443
Epoch: 29, Step: 3260, Loss: 0.171838
Epoch: 29, Step: 3270, Loss: 0.0736655
Epoch: 29, Step: 3280, Loss: 0.0766731
Epoch: 29, Step: 3290, Loss: 0.057415


W0814 20:57:47.979535 139790696757120 saver.py:1134] *******************************************************
W0814 20:57:47.980541 139790696757120 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W0814 20:57:47.981281 139790696757120 saver.py:1136] Consider switching to the more efficient V2 format:
W0814 20:57:47.986128 139790696757120 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W0814 20:57:47.988488 139790696757120 saver.py:1138] now on by default.
W0814 20:57:47.990273 139790696757120 saver.py:1139] *******************************************************


Epoch: 29, Step: 3300, Loss: 0.0745666
Epoch: 29, Step: 3310, Loss: 0.0937294
Epoch: 29, Step: 3320, Loss: 0.483597
Epoch: 29, Step: 3330, Loss: 0.424862
Epoch: 29, Step: 3340, Loss: 0.449543
Epoch: 29, Step: 3350, Loss: 0.179433
Model saved in file: ./save/model.ckpt
Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser


In [0]:
# Predicting Degrees

degrees_predicted = []
for i in range(len(val_xs)):
    full_image = scipy.misc.imread(val_xs[i], mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = sess.run(y,feed_dict={x: [image], keep_prob: 0.7})[0][0] * 180.0 / scipy.pi
    degrees_predicted.append(degrees)

In [15]:
import pandas as pd
data = pd.DataFrame({"degrees":degrees_predicted,"original":[val_ys[i]*180/scipy.pi for i in range(len(val_ys))]})
data

,degrees,original
0,-26.775018,-28.34
1,-36.105442,-28.84
2,-29.432617,-29.75
3,-28.049645,-31.06
4,-23.372687,-32.27
5,-23.521481,-33.48
6,-20.196150,-34.39
7,-17.209048,-35.60
8,-11.020193,-36.50
9,-44.060072,-37.61
